In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

from __future__ import print_function
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import Normalizer
import h5py
from keras import callbacks
from scipy.io import arff
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
import numpy as np
np.random.seed(1337)  # for reproducibility
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding
from keras.layers import LSTM, SimpleRNN, GRU
from keras.datasets import imdb
from keras.utils.np_utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn import metrics
from sklearn.preprocessing import Normalizer
import h5py
from keras import callbacks
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
import h5py


In [ ]:
 #load training dataset
dataTrain = arff.loadarff('/content/drive/My Drive/KDDTrain20MultiClass.arff')
dataset_train = pd.DataFrame(dataTrain[0])
dataTest = arff.loadarff('/content/drive/My Drive/KDDTest21MultiClass.arff')
dataset_test= pd.DataFrame(dataTrain[0])

X_train = dataset_train.iloc[:, :-2].values
Y_train = dataset_train.iloc[:, 41].values

X_test = dataset_test.iloc[:, :-2].values
Y_test = dataset_test.iloc[:, 41].values     

In [ ]:
classes=[0,1,2,3,4,5]
n_classes = 6

In [ ]:
#X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
#X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

X_train.shape

X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

x_train = np.array(X_train)
x_test = np.array(X_test)
y_train1 = np.array(Y_train)
y_test1 = np.array(Y_test)

y_train= to_categorical(y_train1)
y_test= to_categorical(y_test1)

In [ ]:
# 1. define stacked LSTM when dropout is 0.4
model = Sequential()
model.add(LSTM(16,input_dim=40, return_sequences=True))  # try using a GRU instead, for fun
model.add(Dropout(0.4))
model.add(LSTM(16, return_sequences=True))  # try using a GRU instead, for fun
model.add(Dropout(0.4))
model.add(LSTM(16))  # try using a GRU instead, for fun
model.add(Dropout(0.4))
model.add(Dense(6))
model.add(Activation('softmax'))
model.summary()

model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs = 10, batch_size=32, validation_data = (X_test, y_test), verbose=1)

scores = model.evaluate(X_test, y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
# save model and architecture to single file
model.save("model.h5")
print("Saved model to disk")

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, None, 16)          3648      
_________________________________________________________________
dropout (Dropout)            (None, None, 16)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 16)          2112      
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 16)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 16)                2112      
_________________________________________________________________
dropout_2 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 6)                 1

In [ ]:
target_names = ['DoS', 'U2R', 'R2L', 'Probe','Normal','Unknown']

In [ ]:
### train_accuracy
from sklearn.metrics import confusion_matrix
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
import sklearn.metrics as metrics
expected = y_train
predicted = model.predict(X_train)
data = classification_report(expected,np.round(predicted), target_names=target_names)
print(data)

print("Accuracy_Train:",metrics.accuracy_score(expected, np.round(predicted)))
print("Precision_Train:",metrics.precision_score(expected, np.round(predicted), average='weighted'))
print("Recall_Train:",metrics.recall_score(expected, np.round(predicted), average='weighted'))
print("f1-score_Train:",metrics.f1_score(expected, np.round(predicted), average='weighted'))


              precision    recall  f1-score   support

         DoS       0.00      0.00      0.00         0
         U2R       1.00      1.00      1.00      9234
         R2L       0.00      0.00      0.00        11
       Probe       0.00      0.00      0.00       209
      Normal       0.97      0.95      0.96      2289
     Unknown       0.98      0.99      0.99     13449

   micro avg       0.99      0.98      0.98     25192
   macro avg       0.49      0.49      0.49     25192
weighted avg       0.98      0.98      0.98     25192
 samples avg       0.98      0.98      0.98     25192

Accuracy_Train: 0.98193871070181
Precision_Train: 0.9787573011681868
Recall_Train: 0.98193871070181
f1-score_Train: 0.9803202821353763


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1

In [ ]:
###test_accuracy
expected = y_test
predicted = model.predict(X_test)
data = classification_report(expected, np.round(predicted), target_names=target_names)
print(data)

print("Accuracy:",metrics.accuracy_score(expected, np.round(predicted)))
print("Precision:",metrics.precision_score(expected, np.round(predicted), average='weighted'))
print("Recall:",metrics.recall_score(expected, np.round(predicted), average='weighted'))
print("f1-score:",metrics.f1_score(expected, np.round(predicted), average='weighted'))


              precision    recall  f1-score   support

         DoS       0.00      0.00      0.00         0
         U2R       1.00      1.00      1.00      9234
         R2L       0.00      0.00      0.00        11
       Probe       0.00      0.00      0.00       209
      Normal       0.97      0.95      0.96      2289
     Unknown       0.98      0.99      0.99     13449

   micro avg       0.99      0.98      0.98     25192
   macro avg       0.49      0.49      0.49     25192
weighted avg       0.98      0.98      0.98     25192
 samples avg       0.98      0.98      0.98     25192

Accuracy: 0.98193871070181
Precision: 0.9787573011681868
Recall: 0.98193871070181
f1-score: 0.9803202821353763


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1